In [1]:
%pip install --upgrade --quiet pymupdf pillow pytesseract langchain langchain-core langchain_huggingface langchain-google-genai langchain-pinecone langchain-community sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.9/437.9 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 

In [2]:
import fitz  # PyMuPDF
from langchain.document_loaders import PyMuPDFLoader
from langchain_core.documents import Document
import os
import json # For metadata serialization

class EnhancedPyMuPDFLoader(PyMuPDFLoader):
    def load(self):
        """Load documents, extract text, insert image placeholders, and store image metadata."""
        docs_from_super = super().load()  # Provides one Document per page

        processed_docs = []
        pdf_document = fitz.open(self.file_path)

        if len(docs_from_super) != len(pdf_document):
            print(f"Warning: Mismatch in page count between Langchain loader ({len(docs_from_super)}) and fitz ({len(pdf_document)}). Processing based on fitz page count if super().load() is shorter.")

        for i, page_fitz in enumerate(pdf_document):
            # Try to get the corresponding doc from super().load()
            page_doc_from_super = None
            if i < len(docs_from_super) and docs_from_super[i].metadata.get('page') == i:
                page_doc_from_super = docs_from_super[i]

            current_page_content = page_doc_from_super.page_content if page_doc_from_super else page_fitz.get_text("text", sort=True)
            page_metadata = page_doc_from_super.metadata.copy() if page_doc_from_super else {"source": self.file_path, "page": i}

            image_list = page_fitz.get_images(full=True)
            page_specific_image_metadata = {}
            appended_placeholders_text = []

            for img_idx, img_info in enumerate(image_list):
                xref = img_info[0]
                # Create a unique image ID: p{page_num_1_indexed}_i{image_index_1_indexed}
                image_id = f"img_p{i+1}_i{img_idx+1}"
                placeholder = f"[IMAGE: {image_id}]"

                appended_placeholders_text.append(placeholder)

                img_meta = {
                    "image_id": image_id,
                    "page_num": i + 1,  # 1-indexed for user display
                    "image_index_on_page": img_idx + 1,
                    "xref": xref,  # Crucial for later extraction
                    "source_file": self.file_path
                    # Dimensions and format will be added by ImageAnalysisAgent
                }
                page_specific_image_metadata[image_id] = img_meta

            if appended_placeholders_text:
                # Append placeholders at the end of the page content
                current_page_content += "\n\nImage Placeholders on this page:\n" + "\n".join(appended_placeholders_text)

            if page_specific_image_metadata:
                # Store metadata for all images on this page
                page_metadata["images_on_page"] = page_specific_image_metadata

            processed_docs.append(Document(page_content=current_page_content, metadata=page_metadata))

        pdf_document.close()
        return processed_docs

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json # Ensure json is imported here

class ImageAwareTextSplitter(RecursiveCharacterTextSplitter):
    def split_documents(self, documents): # documents are page-level from EnhancedPyMuPDFLoader
        """Split documents while preserving image metadata for relevant chunks."""
        splits = super().split_documents(documents)

        for split in splits:
            # Find the original page document that this split came from
            # RecursiveCharacterTextSplitter preserves metadata from the parent document,
            # including 'page' and 'images_on_page' if they were there.
            original_doc_metadata = split.metadata

            images_on_original_page = original_doc_metadata.get('images_on_page', {})

            if images_on_original_page:
                chunk_specific_images = {}
                for img_id, img_data_from_loader in images_on_original_page.items():
                    # If the placeholder (which was added to page_content by the loader)
                    # is present in this specific text chunk, then this image is relevant to this chunk.
                    if f"[IMAGE: {img_id}]" in split.page_content:
                        # Ensure img_data_from_loader is a dictionary before copying
                        if isinstance(img_data_from_loader, dict):
                            chunk_specific_images[img_id] = img_data_from_loader.copy()
                        else:
                             print(f"Warning: Expected dictionary for image metadata {img_id}, but got {type(img_data_from_loader)}. Skipping.")

                if chunk_specific_images:
                    # Store image metadata relevant *to this chunk* in split.metadata["images"]
                    # This "images" key will be used by the ImageAgent later.
                    # It will be serialized to JSON string before storing in vector DB.
                    split.metadata["images"] = chunk_specific_images # Store as dict here

            # **FIX:** Explicitly remove the original 'images_on_page' key from the chunk's metadata
            # This is crucial because Pinecone does not accept the dictionary format.
            if "images_on_page" in split.metadata:
                del split.metadata["images_on_page"]

        return splits

In [4]:
import pytesseract
from PIL import Image as PILImage
import io
import fitz # PyMuPDF

# For Colab/Linux, Tesseract is usually in path. For others, you might need:
# pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract' # Example

class ImageAnalysisAgent:
    def __init__(self):
        pass

    def _extract_image_details_from_pdf(self, pdf_path, xref):
        """Helper to extract image bytes, format, width, height from PDF using xref."""
        doc = None
        try:
            doc = fitz.open(pdf_path)
            base_image = doc.extract_image(xref) # xref is int
            if not base_image:
                return None, None, None, None, "Extraction failed"

            image_bytes = base_image["image"]
            image_format = base_image["ext"]
            width = base_image.get("width")
            height = base_image.get("height")
            return image_bytes, image_format, width, height, None
        except Exception as e:
            error_message = f"Error extracting image with xref {xref} from {pdf_path}: {e}"
            print(error_message)
            return None, None, None, None, error_message
        finally:
            if doc:
                doc.close()

    def analyze_images(self, image_infos_list):
        """
        Analyzes a list of image metadata dictionaries.
        Each dict in image_infos_list is expected to have 'source_file', 'xref', 'image_id'.
        It adds 'ocr_text', 'image_format', 'width', 'height', 'image_type', 'calculated_relevance'.
        'image_infos_list' comes from ImageAgent.identify_images.
        """
        analyzed_results = []

        for img_info_from_agent in image_infos_list:
            # img_info_from_agent already contains: image_id, page_num, xref, source_file,
            # document_context (text around placeholder), relevance_score (from vector search), placeholder.
            analyzed_img_data = img_info_from_agent.copy()

            pdf_path = analyzed_img_data.get("source_file")
            xref = analyzed_img_data.get("xref")

            if not pdf_path or xref is None:
                print(f"Warning: Missing source_file or xref for image_id {analyzed_img_data.get('image_id')}. Skipping analysis.")
                analyzed_img_data.update({
                    "ocr_text": None, "image_format": None, "width": None, "height": None,
                    "image_type": "unknown", "calculated_relevance": 0.0, "analysis_error": "Missing source/xref"
                })
                analyzed_results.append(analyzed_img_data)
                continue

            image_bytes, img_format, width, height, error = self._extract_image_details_from_pdf(pdf_path, xref)

            analyzed_img_data["image_format"] = img_format
            analyzed_img_data["width"] = width
            analyzed_img_data["height"] = height
            analyzed_img_data["analysis_error"] = error

            ocr_text = None
            if image_bytes and not error:
                try:
                    pil_img = PILImage.open(io.BytesIO(image_bytes))
                    ocr_text = pytesseract.image_to_string(pil_img)
                    ocr_text = ocr_text.strip() if ocr_text else None
                except Exception as e:
                    print(f"Error during OCR for image {analyzed_img_data.get('image_id')}: {e}")
                    analyzed_img_data["analysis_error"] = (analyzed_img_data["analysis_error"] or "") + f"; OCR Error: {e}"
                    ocr_text = None
            analyzed_img_data["ocr_text"] = ocr_text

            # Image Type (Heuristic based on format)
            if img_format:
                fmt_lower = img_format.lower()
                if fmt_lower in ["jpeg", "jpg", "png", "gif", "bmp"]:
                    analyzed_img_data["image_type"] = "raster_graphic" # Could be photo, diagram, screenshot
                elif fmt_lower == "tiff":
                    analyzed_img_data["image_type"] = "tagged_image_file_format" # Often scans or high quality
                elif fmt_lower == "svg":
                     analyzed_img_data["image_type"] = "vector_graphic"
                else:
                    analyzed_img_data["image_type"] = img_format
            else:
                analyzed_img_data["image_type"] = "unknown"

            # Basic Relevance Score (heuristic: larger images or images with some OCR text are more relevant)
            # This is a calculated score, distinct from the vector search relevance_score.
            calculated_relevance = 0.0
            if width and height and width > 50 and height > 50: # Arbitrary size threshold
                calculated_relevance += 0.4
            if ocr_text and len(ocr_text) > 10: # Arbitrary OCR text length threshold
                calculated_relevance += 0.6
            analyzed_img_data["calculated_relevance"] = min(1.0, calculated_relevance) # Cap at 1.0

            analyzed_results.append(analyzed_img_data)

        return analyzed_results

In [5]:
import json # Ensure this is imported in the cell with ImageAgent

class ImageAgent:
    def __init__(self):
        pass

    def identify_images(self, retrieved_docs_with_scores):
        """
        Identify images in the retrieved document chunks.
        Returns: List of image information dictionaries, ready for deeper analysis.
        Each dict contains metadata from the chunk, context, and vector search score.
        """
        images_info_for_analysis = []

        for doc_chunk, score in retrieved_docs_with_scores: # doc_chunk is a text split
            # 'images' metadata in the chunk was populated by ImageAwareTextSplitter
            # and potentially JSON stringified before storing in vector DB
            chunk_images_metadata_raw = doc_chunk.metadata.get("images", {})

            parsed_chunk_images_metadata = {}
            if isinstance(chunk_images_metadata_raw, str):
                try:
                    parsed_chunk_images_metadata = json.loads(chunk_images_metadata_raw)
                except json.JSONDecodeError:
                    print(f"Warning: Could not decode image metadata string for a chunk from page {doc_chunk.metadata.get('page')}.")
                    # parsed_chunk_images_metadata remains {}
            elif isinstance(chunk_images_metadata_raw, dict):
                parsed_chunk_images_metadata = chunk_images_metadata_raw

            if not isinstance(parsed_chunk_images_metadata, dict): # Should be a dict by now
                continue

            for img_id, img_data_from_splitter in parsed_chunk_images_metadata.items():
                # img_data_from_splitter contains: image_id, page_num, image_index_on_page, xref, source_file

                # Create a new dict for this image, copying essential info
                current_image_info = img_data_from_splitter.copy()

                placeholder = f"[IMAGE: {img_id}]" # Reconstruct placeholder to be sure
                current_image_info["placeholder"] = placeholder

                # Extract text context around the placeholder from the chunk's content
                if placeholder in doc_chunk.page_content:
                    placeholder_pos = doc_chunk.page_content.find(placeholder)
                    start_pos = max(0, placeholder_pos - 150) # More context
                    end_pos = min(len(doc_chunk.page_content), placeholder_pos + len(placeholder) + 150)
                    current_image_info["document_context"] = doc_chunk.page_content[start_pos:end_pos]
                else:
                    # This case should ideally not happen if ImageAwareTextSplitter worked correctly
                    # and placeholder was indeed in the chunk that references this image.
                    current_image_info["document_context"] = "Placeholder not found in chunk context."

                current_image_info["vector_search_relevance_score"] = score # Relevance of the text chunk

                images_info_for_analysis.append(current_image_info)

        return images_info_for_analysis

In [6]:
class Phase2Orchestrator:
    def __init__(self, retrieval_agent, image_agent, image_analysis_agent):
        self.retrieval_agent = retrieval_agent
        self.image_agent = image_agent # Identifies images in retrieved docs + context
        self.image_analysis_agent = image_analysis_agent # Performs OCR, etc.

    def process_query(self, query, k=3):
        # 1. Retrieve text chunks
        retrieved_docs_with_scores = self.retrieval_agent.retrieve(query, k=k)

        # 2. Identify image metadata associated with these chunks & get local text context
        # Output: list of dicts, each is an image's info (incl. context, placeholder, xref, source_file, vector_score)
        images_identified_in_chunks = self.image_agent.identify_images(retrieved_docs_with_scores)

        # 3. Perform deeper analysis (OCR, type, etc.) on these identified images
        analyzed_images_complete_info = []
        if images_identified_in_chunks:
             analyzed_images_complete_info = self.image_analysis_agent.analyze_images(images_identified_in_chunks)

        return {
            "query": query,
            "retrieved_docs_with_scores": retrieved_docs_with_scores, # List of (Document, score)
            "analyzed_image_info": analyzed_images_complete_info # List of dicts with full analysis
        }

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
# (Existing imports)

class EnhancedResponseGenerator:
    def __init__(self, orchestrator): # orchestrator is now Phase2Orchestrator
        self.orchestrator = orchestrator
        # Ensure GOOGLE_API_KEY_1 is correctly loaded via userdata
        self.llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", api_key=userdata.get("GOOGLE_API_KEY_1"))

        # Updated prompt template for Phase 2
        self.prompt_template = PromptTemplate.from_template(
            """You are an AI assistant answering questions based on provided document excerpts and image analyses.
Use only the information given in the 'Document Context' and 'Image Analysis Details'.

Document Context:
{context}

Image Analysis Details:
{image_analysis_details}

Question: {query}

Based on all the provided information, answer the question.
- If OCR text from an image is available and relevant, incorporate it into your answer.
- If an image is described (e.g., type, context) but has no OCR, acknowledge its likely content based on the description if relevant.
- Refer to images using their placeholders like "[IMAGE: img_pX_iY]" when discussing them.
- If the information is insufficient to answer, clearly state that.
Provide a comprehensive and accurate answer.
"""
        )

    def generate_response(self, query):
        # results will contain 'retrieved_docs_with_scores' and 'analyzed_image_info'
        results = self.orchestrator.process_query(query)

        # Format document context
        context_str = "\n\n---\n\n".join([
            f"Excerpt from page {doc.metadata.get('page', 'N/A')+1} (Chunk Relevance: {score:.2f}):\n{doc.page_content}"
            for doc, score in results["retrieved_docs_with_scores"]
        ])

        # Format image analysis details
        image_analysis_str = "No specific images identified or analyzed for this query context.\n"
        if results["analyzed_image_info"]:
            image_analysis_str = "The following image analyses are relevant to the document context:\n"
            for i, img_data in enumerate(results["analyzed_image_info"]):
                image_analysis_str += f"\n--- Image Analysis {i+1} ({img_data.get('placeholder', 'Unknown Placeholder')}) ---\n"
                image_analysis_str += f"  Source: Page {img_data.get('page_num', 'N/A')}, Image {img_data.get('image_index_on_page', 'N/A')} in PDF\n"
                image_analysis_str += f"  Detected Type: {img_data.get('image_type', 'N/A')}\n"
                image_analysis_str += f"  Dimensions: {img_data.get('width')}x{img_data.get('height')}px (Format: {img_data.get('image_format', 'N/A')})\n"
                image_analysis_str += f"  Calculated Image Relevance: {img_data.get('calculated_relevance', 0.0):.2f}\n"

                ocr = img_data.get("ocr_text")
                if ocr:
                    image_analysis_str += f"  OCR Text from Image: \"{ocr[:700]}\"{(len(ocr)>700) * '...'}\n" # Truncate long OCR
                else:
                    image_analysis_str += "  OCR Text from Image: Not available or not significant.\n"

                image_analysis_str += f"  Text Context from Document around image: \"...{img_data.get('document_context', 'N/A')}...\"\n"
                if img_data.get("analysis_error"):
                     image_analysis_str += f"  Analysis Note: {img_data.get('analysis_error')}\n"
            image_analysis_str += "\n"

        # Prepare input for the LLM
        llm_input_dict = {
            "query": query,
            "context": context_str,
            "image_analysis_details": image_analysis_str
        }

        # Invoke the LLM
        # The invoke method for ChatGoogleGenerativeAI expects a string, ChatPromptValue, or list of messages.
        # Formatting the prompt template gives a string.
        formatted_prompt = self.prompt_template.format(**llm_input_dict)
        response = self.llm.invoke(formatted_prompt)

        return response.content

In [8]:
# Ensure all necessary imports are at the top of your notebook or script
from google.colab import userdata # For API keys
import json
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

In [18]:
# --- Cell 2: Define PDF Path and Load API Keys ---
pdf_file_path = "/content/GPU.pdf" # Make sure this file exists
PINECONE_API_KEY = userdata.get("PINECONE_API_KEY")
GOOGLE_API_KEY_1 = userdata.get("GOOGLE_API_KEY_1") # Used in EnhancedResponseGenerator

# --- Cell 3: Instantiate Components ---

# 1. Enhanced Loader
loader = EnhancedPyMuPDFLoader(pdf_file_path)
documents = loader.load() # Page-level docs with 'images_on_page' metadata and placeholders in text

# 2. Image-Aware Splitter
splitter = ImageAwareTextSplitter(chunk_size=500, chunk_overlap=50, length_function=len)
chunks = splitter.split_documents(documents) # Chunks with 'images' metadata (img_id -> {xref, source_file, ...})

# 3. Serialize 'images' metadata for Pinecone (if it's a dictionary)
for chunk in chunks:
    # Check for the 'images' key (populated by the splitter) and ensure it's a dictionary
    if "images" in chunk.metadata and isinstance(chunk.metadata["images"], dict):
        try:
            # Serialize the dictionary to a JSON string
            chunk.metadata["images"] = json.dumps(chunk.metadata["images"])
        except TypeError as e:
            print(f"Error serializing image metadata for chunk on page {chunk.metadata.get('page')}: {e}. Removing problematic key.")
            # Remove the key if serialization fails to prevent the Pinecone error
            del chunk.metadata["images"]
    # Ensure 'images_on_page' is NOT present in the metadata being sent to Pinecone
    # This should be handled by the ImageAwareTextSplitter, but a final check doesn't hurt.
    if "images_on_page" in chunk.metadata:
         print(f"Warning: 'images_on_page' found in chunk metadata before Pinecone upload for chunk on page {chunk.metadata.get('page')}. This should have been removed by the splitter. Removing it now.")
         del chunk.metadata["images_on_page"]

In [19]:
image_chunks = [chunk for chunk in chunks if "images" in chunk.metadata]

In [20]:
import json

for chunk in image_chunks:
    if "images" in chunk.metadata and isinstance(chunk.metadata["images"], dict):
        chunk.metadata["images"] = json.dumps(chunk.metadata["images"])

In [15]:
# 4. Embedding Model
embedding_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct",
    model_kwargs={'device': 'cpu'} # Use 'cuda' if GPU is available and configured
)
embedding_dimensions = len(embedding_model.embed_query("test")) # From Phase 1

In [21]:
# 5. Pinecone Setup (assuming index 'multiagent-rag' exists and is configured)
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "multiagent-rag"
pinecone_index = pc.Index(index_name)
# If you need to re-create or ensure dimensions:
if index_name not in [index['name'] for index in pc.list_indexes()]:
    # Create index if it doesn't exist
    pc.create_index(
        name=index_name,
        dimension=embedding_dimensions,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Index '{index_name}' created.")
    pinecone_index = pc.Index(index_name)
else:
    pinecone_index = pc.Index(index_name)
    print(f"Index '{index_name}' already exists and has been initialized")

vector_store = PineconeVectorStore(index=pinecone_index, embedding=embedding_model)

# IMPORTANT: If you've changed the structure of chunks or their metadata,
# you might need to clear the old index or use a new one, and re-add documents.
print(f"Adding {len(chunks)} chunks to the vector store...")
vector_store.add_documents(chunks)
print("Finished adding chunks.")

Index 'multiagent-rag' already exists and has been initialized
Adding 100 chunks to the vector store...
Finished adding chunks.


In [22]:
# --- Cell 4: Instantiate Phase 2 Agents and Orchestrator ---
# (Assuming vector_store is initialized and populated from above or previous run)
# Ensure RetrievalAgent is defined as in Phase 1, using the vector_store
class RetrievalAgent:
    def __init__(self):
        self.embeddings = embedding_model
        self.vector_store = vector_store # Make sure vector_store is initialized
    def retrieve(self, query, k=3):
        return self.vector_store.similarity_search_with_score(query, k=k)

In [27]:
retrieval_agent = RetrievalAgent() # Assumes vector_store is globally available or passed in
image_agent_p2 = ImageAgent() # Phase 2 version (or refined Phase 1 version)
image_analysis_agent_p2 = ImageAnalysisAgent()

orchestrator_p2 = Phase2Orchestrator(
    retrieval_agent,
    image_agent_p2,
    image_analysis_agent_p2
)

generator_p2 = EnhancedResponseGenerator(orchestrator_p2)

# --- Cell 5: Test Phase 2 System ---
question_p2 = "Explain me Matrix-matrix product –tiling and shared memory usage"
print(f"\nProcessing P2 Question: {question_p2}\n")

response_p2_content = generator_p2.generate_response(question_p2)
print("\n--- Phase 2 LLM Response ---")
print(response_p2_content)


Processing P2 Question: Explain me Matrix-matrix product –tiling and shared memory usage


--- Phase 2 LLM Response ---
The provided text mentions "Matrix-matrix product – tiling and shared memory usage" and "Matrix-matrix product – tiling and shared memory (cont)" from the NVIDIA-UIUC GPU teaching kit and the NVIDIA CUDA Programming Guide.  The text notes that in matrix multiplication with tiling, the tile size should match the block size and fit within shared memory.  However, no further details on the implementation or specifics of tiling and shared memory usage are available in the provided text.  The image analyses are unhelpful as they contain no relevant OCR text and are described as having unknown types and low relevance.  Therefore, a complete explanation of matrix-matrix product – tiling and shared memory usage cannot be provided based solely on the given information.


In [28]:
# --- Optional: For Debugging ---
print("\n--- Debugging Orchestrator P2 Output ---")
debug_results_p2 = orchestrator_p2.process_query(question_p2, k=2)
print(f"Query: {debug_results_p2['query']}")
print(f"\nRetrieved Docs ({len(debug_results_p2['retrieved_docs_with_scores'])}):")
for i, (doc, score) in enumerate(debug_results_p2['retrieved_docs_with_scores']):
    print(f"  Doc {i+1} (Page {doc.metadata.get('page', 'N/A')+1}, Score: {score:.3f}):")
    print(f"    Content snippet: {doc.page_content[:200]}...")
    if "images" in doc.metadata: # This would be the JSON string from Pinecone
        print(f"    Raw images metadata: {doc.metadata['images'][:100]}...")


print(f"\nAnalyzed Image Info ({len(debug_results_p2['analyzed_image_info'])}):")
for i, img_info in enumerate(debug_results_p2['analyzed_image_info']):
    print(f"  Image {i+1}: {img_info.get('placeholder')}")
    print(f"    Page: {img_info.get('page_num')}, XRef: {img_info.get('xref')}")
    print(f"    Type: {img_info.get('image_type')}, Format: {img_info.get('image_format')}, Calculated Relevance: {img_info.get('calculated_relevance')}")
    print(f"    OCR: '{str(img_info.get('ocr_text'))[:100]}...'")
    print(f"    Document Context: '{img_info.get('document_context', 'N/A')[:100]}...'")
    if img_info.get('analysis_error'):
        print(f"    Analysis Error: {img_info.get('analysis_error')}")


--- Debugging Orchestrator P2 Output ---
Query: Explain me Matrix-matrix product –tiling and shared memory usage

Retrieved Docs (2):
  Doc 1 (Page 30.0, Score: 0.941):
    Content snippet: Matrix-matrix product – tiling and shared memory (cont) 
Source: GPU teaching kit, NVIDIA-UIUC...
    Raw images metadata: {"img_30_1": {"image_id": "img_30_1", "page_num": 30, "position": 1, "source_file": "/content/GPU.pd...
  Doc 2 (Page 30.0, Score: 0.941):
    Content snippet: Matrix-matrix product – tiling and shared memory (cont) 
Source: GPU teaching kit, NVIDIA-UIUC...
    Raw images metadata: {"img_30_1": {"image_id": "img_30_1", "page_num": 30, "position": 1, "source_file": "/content/GPU.pd...

Analyzed Image Info (2):
  Image 1: [IMAGE: img_30_1]
    Page: 30, XRef: None
    Type: unknown, Format: None, Calculated Relevance: 0.0
    OCR: 'None...'
    Document Context: 'Placeholder not found in chunk context....'
    Analysis Error: Missing source/xref
  Image 2: [IMAGE: img_30_1]
    P